# UK - Anbindung Paid Themen 

* Um was handelt es sich hier  (Kurzbeschreibung Inhalt):
- Die Paid Data Transformatation von LinkedIn


---
* QUELLEN:  
- datif_dz_{}.`02_cleaned_uk_linkedin`.`020_li_paid_current_view`


* ZIEL:  
- datif_pz_uk_{}.03_transformed.linkedin_paid_daily
- datif_pz_uk_{}.03_transformed.linkedin_paid_total

---
* Versionen (aktuelle immer oben):
- 21.08.2025 Justin Stange-Heiduk: init
- 17.09.2025 Justin Stange-Heiduk: Ad OrganicInfo


# 1. Initialnotebooks & Libraries

In [0]:
%run ../../common/nb_init

In [0]:
target_schema_name = "03_transformed"
target_path = "funnel"

## 03-transformed

### Daily

In [0]:
spark.sql(f"""
    CREATE OR REPLACE VIEW datif_pz_uk_{env}.03_transformed.linkedin_paid_daily AS
    SELECT
        FIRST(CAST(DATE_FORMAT(Date, 'yyyy-MM-dd') AS DATE)) AS Date,
        FIRST(CAST(Creative_ID__LinkedIn AS STRING)) AS AdID,
        FIRST(CAST(Campaign__LinkedIn AS STRING)) AS CampaignName,
        FIRST(CAST(Campaign_Group__LinkedIn AS STRING)) AS AdSetName,
        FIRST(CAST(Creative_Name__LinkedIn AS STRING)) AS AdName,
        FIRST(
            CASE 
                WHEN Creative_Name__LinkedIn = Creative_ID__LinkedIn
                    THEN 'Kein Dark Post'
                ELSE 'Dark Post'
            END
        ) AS OrganicInfo,
        FIRST(CAST(Creative_Text__LinkedIn AS STRING)) AS Text,
        FIRST(CAST(Creative_Title__LinkedIn AS STRING)) AS Title,
        FIRST(CAST(Click_URL__LinkedIn AS STRING)) AS Click_URL,
        FIRST(CAST(Campaign_Objective__LinkedIn AS STRING)) AS Campaign_Objective,
        FIRST(CAST(Ad_Type__LinkedIn AS STRING)) AS PostType,
        SUM(COALESCE(ROUND(CAST(Spend__LinkedIn AS DOUBLE), 2), 0)) AS Amount_Spend,
        SUM(COALESCE(CAST(Clicks__LinkedIn AS INTEGER), 0)) AS Clicks,
        SUM(COALESCE(CAST(Impressions__LinkedIn AS INTEGER), 0)) AS Impressions,
        SUM(COALESCE(CAST(Text_URL_Clicks__LinkedIn AS INTEGER), 0)) AS Link_Clicks,
        SUM(COALESCE(CAST(Conversions__LinkedIn AS INTEGER), 0)) AS Conversions,
        SUM(COALESCE(CAST(Post_Reactions__LinkedIn AS INTEGER), 0)) AS Reactions,
        SUM(COALESCE(CAST(Download_Clicks__LinkedIn AS INTEGER), 0)) AS Post_Saves,
        SUM(COALESCE(CAST(Shares__LinkedIn AS INTEGER), 0)) AS Post_Shares,
        SUM(COALESCE(CAST(Comments__LinkedIn AS INTEGER), 0)) AS Comments,
        SUM(COALESCE(CAST(Likes__LinkedIn AS INTEGER), 0)) AS Likes,
        SUM(COALESCE(CAST(Video_Views__LinkedIn AS INTEGER), 0)) AS Video_View,
        SUM(
            COALESCE(0.4 * CAST(Comments__LinkedIn AS DOUBLE), 0) +
            COALESCE(0.3 * CAST(Shares__LinkedIn AS DOUBLE), 0) +
            COALESCE(0.2 * CAST(Clicks__LinkedIn AS DOUBLE), 0) +
            COALESCE(0.1 * CAST(Likes__LinkedIn AS DOUBLE), 0)
        ) AS WeightedEngagements,
        SUM(
            COALESCE(CAST(Comments__LinkedIn AS INTEGER), 0) +
            COALESCE(CAST(Shares__LinkedIn AS INTEGER), 0) +
            COALESCE(CAST(Clicks__LinkedIn AS INTEGER), 0) +
            COALESCE(CAST(Likes__LinkedIn AS INTEGER), 0)
        ) AS Engagements
    FROM datif_dz_{env}.`02_cleaned_uk_linkedin`.`020_li_paid_current_view`
    GROUP BY Date, Creative_Title__LinkedIn
""")

### Total

In [0]:
%python
df_daily = spark.read.table(f"datif_pz_uk_{env}.03_transformed.linkedin_paid_daily")

df_total = (
    df_daily.groupBy("AdID").agg(
        F.first("AdID").alias("ID"),
        F.first("OrganicInfo").alias("OrganicInfo"),
        F.min("Date").alias("CreatedDate").cast('date'),
        F.first("CampaignName").alias("CampaignName"),
        F.first("AdSetName").alias("AdSetName"),
        F.first("AdName").alias("AdName"),
        F.first("Text").alias("Text"),
        F.first("Click_URL").alias("Click_URL"),
        F.round(F.sum("Amount_Spend"),2).alias("TotalSpend").cast('Double'),
        F.sum("Impressions").alias("TotalImpressions").cast('integer'),
        F.sum("Clicks").alias("TotalClicks").cast('integer'),
        F.sum("Link_Clicks").alias("TotalLinkClicks").cast('integer'),
        F.sum("Post_Saves").alias("TotalPostSaves").cast('integer'),
        F.sum("Post_Shares").alias("TotalPostShares").cast('integer'),
        F.sum("Comments").alias("TotalPostComments").cast('integer'),
        F.sum("Likes").alias("TotalLikes").cast('integer'),
        F.sum("Video_View").alias("TotalVideoView").cast('integer'),
        F.sum("WeightedEngagements").alias("TotalWeightedEngagements").cast('integer'),
        F.sum("Engagements").alias("TotalEngagements").cast('integer') 
    ).drop("AdID")
    .withColumn(
        "EngagementRatings",
        F.coalesce(
            F.round(
                F.when(F.col("TotalImpressions") == 0, 0).otherwise(
                    F.col("TotalWeightedEngagements") / F.col("TotalImpressions")
                ), 3
            ),
            F.lit(0)
        )
    )
    .withColumn(
        "CPM",
        F.coalesce(
            F.round(
                F.when(F.col("TotalImpressions") == 0, 0).otherwise(
                    (F.col("TotalSpend") / F.col("TotalImpressions")) * 1000
                ), 4
            ),
            F.lit(0)
        )
    )
    .withColumn(
        "CPC",
        F.coalesce(
            F.round(
                F.when(F.col("TotalClicks") == 0, 0).otherwise(
                    F.col("TotalSpend") / F.col("TotalClicks")
                ), 4
            ),
            F.lit(0)
        )
    )
    .withColumn(
        "CTR",
        F.coalesce(
            F.round(
                F.when(F.col("TotalImpressions") == 0, 0).otherwise(
                    F.col("TotalClicks") / F.col("TotalImpressions")
                ), 4
            ),
            F.lit(0)
        )
    )
    .withColumn(
        "CPV",
        F.coalesce(
            F.round(
                F.when(F.col("TotalVideoView") == 0, 0).otherwise(
                    F.col("TotalSpend") / F.col("TotalVideoView")
                ), 4
            ),
            F.lit(0)
        )
    )
    .withColumn(
        "VideoViewRate",
        F.coalesce(
            F.round(
                F.when(F.col("TotalImpressions") == 0, 0).otherwise(
                    F.col("TotalVideoView") / F.col("TotalImpressions")
                ), 4
            ),
            F.lit(0)
        )
    )
    .drop("AdSetName", "Click_URL", "Clicks", "Link_Clicks", "Post_Saves", "Post_Shares", "Comments", "Likes", "Video_View")
)

fn_overwrite_table(df_source=df_total, target_schema_name=target_schema_name, target_table_name="linkedin_paid_total", target_path=target_path)